In [1]:
import pandas as pd
import numpy as np

# Input

In [2]:
ICON = '../../experiments/exp10x/run1/cat/tables/abf5835_data_file_s1.csv'

In [3]:
DF = '../../experiments/exp10x/run1/cat/eval_clonotypes/valid_ct.csv'

In [4]:
TB = '../../experiments/exp10x/run1/cat/tables/tcr_barcode.csv'
TCR = '../../experiments/exp10x/run1/tcr/augmented/tcr.clean.augmented.csv'

In [5]:
CLONO1 = '../../experiments/exp10x/raw/vdj_v1_hs_aggregated_donor1_clonotypes.csv'
CLONO2 = '../../experiments/exp10x/raw/vdj_v1_hs_aggregated_donor2_clonotypes.csv'
CLONO3 = '../../experiments/exp10x/raw/vdj_v1_hs_aggregated_donor3_clonotypes.csv'
CLONO4 = '../../experiments/exp10x/raw/vdj_v1_hs_aggregated_donor4_clonotypes.csv'

In [6]:
DONOR1 = '../../experiments/exp10x/raw/vdj_v1_hs_aggregated_donor1_all_contig_annotations.csv'
CONTIG = '../../experiments/exp10x/run1/tcr/cellranger_tot/outs/multi/vdj_t/all_contig_annotations.csv'

# Output

# Load

In [7]:
tb = pd.read_csv(TB, low_memory=False)
tcr = pd.read_csv(TCR, low_memory=False)

In [8]:
contig = pd.read_csv(CONTIG, low_memory=False)

In [9]:
clono1 = pd.read_csv(CLONO1)
clono2 = pd.read_csv(CLONO2)
clono3 = pd.read_csv(CLONO3)
clono4 = pd.read_csv(CLONO4)

In [10]:
icon_all = pd.read_csv(ICON,
                   usecols=['Cell barcode','Donor','pMHC','beta_v_gene','beta_j_gene','beta_cdr3','alpha_v_gene','alpha_j_gene','alpha_cdr3','UMI','corrected dex. Sig.','normalized dex. Sig.'],
                   encoding='unicode_escape')

In [11]:
df = pd.read_csv(DF, low_memory=False)

In [12]:
donor1 = pd.read_csv(DONOR1, low_memory=False)

In [13]:
def get_clono():
    if smpl == 's1':
        return clono1
    elif smpl == 's2':
        return clono2
    elif smpl == 's3':
        return clono3
    else:
        return clono4

# Prep

In [14]:
gem_df = contig.barcode.str.split('-s', expand=True)
contig['gem'] = gem_df[0]
contig['gem_spl'] = gem_df[0].str.split('-', expand=True)[0]
contig['sample_id'] = 's' + gem_df[1]

### DF

In [13]:
gem_df = df.gem.str.split('-s', expand=True)
#df.gem = gem_df[0]
df['gem_spl'] = gem_df[0].str.split('-', expand=True)[0]
df.sample_id = 's' + gem_df[1]

In [15]:
sample2donor = {'s1':'Donor 1',
                's2':'Donor 2',
                's3':'Donor 3',
                's4':'Donor 4'}

In [15]:
df['donor'] = df.sample_id.map(sample2donor)

In [16]:
haplotypes = {'s1': ['A0201','A1101','B3501'],
              's2': ['A0201','A0101','B0801'],
              's3': ['A2402','A2902','B3502','B4403'],
              's4': ['A0301','B0702','B5701']}

In [17]:
df.HLA_cd8 = df.sample_id.map(haplotypes)

In [26]:
df['v_alpha_match'] = df.v_gene_TRA == df.alpha_v_gene.str.replace('/','')
df['j_alpha_match'] = df.j_gene_TRA == df.alpha_j_gene.str.replace('/','')
df['v_beta_match'] = df.v_gene_TRB == df.beta_v_gene.str.replace('/','')
df['j_beta_match'] = df.j_gene_TRB == df.beta_j_gene.str.replace('/','')

### ICON

In [17]:
# # Fix GEM IDs
# V_idx = icon[icon.Donor == 'Donor V'].index
# icon.loc[V_idx, 'Cell barcode'] = icon.loc[V_idx, 'Cell barcode'].str.replace('.1','-1', regex=False)

icon = icon_all[icon_all.Donor != 'Donor V'].copy()

In [18]:
gem_df = icon['Cell barcode'].str.split('.', expand=True)

icon['gem'] = gem_df[1]
icon['gem_spl'] = gem_df[1].str.split('-',expand=True)[0]
icon['sample_id'] = gem_df[0].str.replace('DV?','s', regex=True)

In [19]:
icon['HLA_cd8'] = icon.sample_id.map(haplotypes)

In [20]:
icon_pMHC = icon.pMHC.str.split('_', n=2, expand=True)
icon['peptide_HLA'] = icon_pMHC[1] +' '+ icon_pMHC[0].str.replace('*','', regex=False)
icon['HLA_mhc'] = icon_pMHC[0].str.replace('*','', regex=False)

In [21]:
icon.peptide_HLA.value_counts()

RAKFKQLL B0801      18467
GILGFVFTL A0201      7544
IVTDFSVIK A1101      5438
KLGGALQAK A0301      3899
AVFDRKSDAK A1101     1575
GLCTLVAML A0201      1411
ELAGIGILTV A0201      209
LLDFVRFMGV A0201      130
FLYALALLL A0201       118
FLRGRAYGL B0801        40
RLRAEAQVK A0301        30
LLFGYPVYV A0201        28
CYTWNQMNL A2402        16
AYAQKIFKI A2402        12
RTLNAWVKV A0201        12
YLLEMLWRL A0201        10
IMDQVPFSV A0201        10
MLDLQPETT A0201         8
KVLEYVIKV A0201         6
KTWGQYWQV A0201         6
IPSINVHHY B3501         5
SLLMWITQV A0201         4
RMFPNAPYL A0201         4
QPRAPIRPI B0702         4
YLNDHLEPWI A0201        3
QYDPVAALF A2402         3
FLASKIGRLV A0201        3
ELRRKMMYM B0801         3
SLFNTVATLY A0201        3
RPPIFIRRL B0702         3
SLFNTVATL A0201         2
RIAAWMATY A0301         1
KVAELVHFL A0201         1
SLYNTVATLY A0201        1
NLVPMVATV A0201         1
Name: peptide_HLA, dtype: int64

# Merge

In [22]:
tmp = pd.merge(df[df.columns[1:104]], icon, # added [df.columns[1:104]]
               left_on=['gem_spl','sample_id'],
               right_on=['gem_spl','sample_id'], # 270 gem_spl (all donor 2) do not exist in original data.
               how='left', suffixes=['','_icon'])

In [23]:
tmp['ca_match'] = tmp.cdr3_TRA == tmp.alpha_cdr3
tmp['cb_match'] = tmp.cdr3_TRB == tmp.beta_cdr3
tmp['va_match'] = tmp.v_gene_TRA == tmp.alpha_v_gene.str.replace('/','')
tmp['ja_match'] = tmp.j_gene_TRA == tmp.alpha_j_gene.str.replace('/','')
tmp['vb_match'] = tmp.v_gene_TRB == tmp.beta_v_gene.str.replace('/','')
tmp['jb_match'] = tmp.j_gene_TRB == tmp.beta_j_gene.str.replace('/','')

In [24]:
tmp['peptide_HLA_match'] = tmp.peptide_HLA == tmp.peptide_HLA_icon

In [25]:
tmp.sort_values(by=['gem_spl','ca_match','cb_match','va_match','vb_match','ja_match','jb_match','peptide_HLA_match'], inplace=True)
# 2039 duplicates --> 1033 GEMs with had multiple entries in ICON
tmp.drop_duplicates(['gem','sample_id'], keep='last', inplace=True) #.dropna(subset=['gem_icon']).gem.nunique()#.to_csv(WHLA_OUT, index=False)

In [26]:
# Overwrites the duplicates from merge
tmp.loc[tmp.duplicated(subset=['Cell barcode'], keep='last') & ~tmp.gem_icon.isna(), 'gem_icon'] = np.nan

In [27]:
# Redefine clonotypes
clono = tmp[~tmp.gem_icon.isna()].groupby(['beta_cdr3','beta_v_gene','beta_j_gene','alpha_cdr3','alpha_v_gene','alpha_j_gene']).size().sort_values(ascending=False).to_frame()
clono['ct'] = np.arange(1, len(clono)+1)
#tmp['ct_atrap'] = tmp.ct
tmp['ct_icon'] = tmp.set_index(['beta_cdr3','beta_v_gene','beta_j_gene','alpha_cdr3','alpha_v_gene','alpha_j_gene']).index.map(clono.ct)

In [26]:
tmp.loc[tmp.duplicated(subset=['Cell barcode'], keep='last') & ~tmp.gem_icon.isna(), #tmp.duplicated(subset=['Cell barcode'], keep=False) & ~tmp.gem_icon.isna(),
        ['gem','gem_icon','Cell barcode','clonotype','ct','peptide_HLA','peptide_HLA_icon','va_match','vb_match','ja_match','jb_match']]#.sort_values(by=['gem','Cell barcode','va_match','vb_match','ja_match','jb_match'])

,gem,gem_icon,Cell barcode,clonotype,ct,peptide_HLA,peptide_HLA_icon,va_match,vb_match,ja_match,jb_match


In [27]:
tmp.loc[tmp.duplicated(subset=['Cell barcode'], keep=False) & ~tmp.gem_icon.isna(), #tmp.duplicated(subset=['Cell barcode'], keep=False) & ~tmp.gem_icon.isna(),
        ['gem','gem_icon','Cell barcode','clonotype','ct','peptide_HLA','peptide_HLA_icon','va_match','vb_match','ja_match','jb_match']].head(20)#.sort_values(by=['gem','Cell barcode','va_match','vb_match','ja_match','jb_match'])

,gem,gem_icon,Cell barcode,clonotype,ct,peptide_HLA,peptide_HLA_icon,va_match,vb_match,ja_match,jb_match
61,AAACCTGAGGCCCGTT-22-s2,AAACCTGAGGCCCGTT-17,D2.AAACCTGAGGCCCGTT-17,clonotype30394,11681,RAKFKQLL B0801,RAKFKQLL B0801,False,True,False,True
138,AAACCTGCACGCCAGT-28-s2,AAACCTGCACGCCAGT-16,D2.AAACCTGCACGCCAGT-16,clonotype30620,30620,GILGFVFTL A0201,GILGFVFTL A0201,True,True,True,True
165,AAACCTGCAGTATGCT-11-s2,AAACCTGCAGTATGCT-26,D2.AAACCTGCAGTATGCT-26,clonotype30546,30546,RAKFKQLL B0801,RAKFKQLL B0801,True,True,True,True
222,AAACCTGGTCTAGTGT-25-s2,AAACCTGGTCTAGTGT-27,D2.AAACCTGGTCTAGTGT-27,clonotype30341,685,RAKFKQLL B0801,RAKFKQLL B0801,True,True,True,True
231,AAACCTGGTCTTGATG-32-s2,AAACCTGGTCTTGATG-30,D2.AAACCTGGTCTTGATG-30,clonotype30341,685,RAKFKQLL B0801,RAKFKQLL B0801,True,True,True,True
284,AAACCTGTCACGATGT-16-s1,AAACCTGTCACGATGT-20,D1.AAACCTGTCACGATGT-20,clonotype1,1,IVTDFSVIK A1101,IVTDFSVIK A1101,True,True,True,True
300,AAACCTGTCCAAGTAC-28-s1,AAACCTGTCCAAGTAC-16,D1.AAACCTGTCCAAGTAC-16,clonotype1362,1362,GILGFVFTL A0201,GILGFVFTL A0201,True,True,True,True
423,AAACGGGAGCGATGAC-30-s2,AAACGGGAGCGATGAC-14,D2.AAACGGGAGCGATGAC-14,clonotype30342,745,RAKFKQLL B0801,RAKFKQLL B0801,True,True,True,True
444,AAACGGGAGGAGTTTA-5-s2,AAACGGGAGGAGTTTA-1,D2.AAACGGGAGGAGTTTA-1,clonotype30341,685,RAKFKQLL B0801,RAKFKQLL B0801,True,True,True,True
582,AAACGGGCATGGTCTA-34-s1,AAACGGGCATGGTCTA-28,D1.AAACGGGCATGGTCTA-28,clonotype1,1,IVTDFSVIK A1101,IVTDFSVIK A1101,True,True,True,True


In [71]:
tmp.shape

(181913, 126)

In [160]:
tmp.gem.nunique()

181913

In [76]:
tmp[~tmp.gem_icon.isna()].peptide_HLA_icon.value_counts() # & (tmp.peptide_HLA_match == False)

RAKFKQLL B0801      17778
GILGFVFTL A0201      7259
IVTDFSVIK A1101      5272
KLGGALQAK A0301      3693
AVFDRKSDAK A1101     1548
GLCTLVAML A0201      1366
ELAGIGILTV A0201      202
LLDFVRFMGV A0201      129
FLYALALLL A0201       117
FLRGRAYGL B0801        40
RLRAEAQVK A0301        30
LLFGYPVYV A0201        26
CYTWNQMNL A2402        16
AYAQKIFKI A2402        12
RTLNAWVKV A0201        11
IMDQVPFSV A0201        10
YLLEMLWRL A0201         9
KVLEYVIKV A0201         6
MLDLQPETT A0201         6
KTWGQYWQV A0201         6
IPSINVHHY B3501         5
SLLMWITQV A0201         4
RMFPNAPYL A0201         4
QPRAPIRPI B0702         4
SLFNTVATLY A0201        3
YLNDHLEPWI A0201        3
FLASKIGRLV A0201        3
QYDPVAALF A2402         3
SLFNTVATL A0201         2
RPPIFIRRL B0702         2
ELRRKMMYM B0801         2
NLVPMVATV A0201         1
SLYNTVATLY A0201        1
RIAAWMATY A0301         1
Name: peptide_HLA_icon, dtype: int64

In [79]:
tmp[~tmp.gem_icon.isna()].groupby('peptide_HLA_icon').ct_icon.nunique()

peptide_HLA_icon
AVFDRKSDAK A1101      85
AYAQKIFKI A2402       12
CYTWNQMNL A2402       16
ELAGIGILTV A0201     202
ELRRKMMYM B0801        2
FLASKIGRLV A0201       3
FLRGRAYGL B0801       12
FLYALALLL A0201       18
GILGFVFTL A0201      863
GLCTLVAML A0201       69
IMDQVPFSV A0201       10
IPSINVHHY B3501        5
IVTDFSVIK A1101      133
KLGGALQAK A0301     2685
KTWGQYWQV A0201        6
KVLEYVIKV A0201        6
LLDFVRFMGV A0201      16
LLFGYPVYV A0201       14
MLDLQPETT A0201        6
NLVPMVATV A0201        1
QPRAPIRPI B0702        4
QYDPVAALF A2402        3
RAKFKQLL B0801       457
RIAAWMATY A0301        1
RLRAEAQVK A0301       30
RMFPNAPYL A0201        4
RPPIFIRRL B0702        2
RTLNAWVKV A0201       11
SLFNTVATL A0201        2
SLFNTVATLY A0201       3
SLLMWITQV A0201        4
SLYNTVATLY A0201       1
YLLEMLWRL A0201        7
YLNDHLEPWI A0201       3
Name: ct_icon, dtype: int64

In [83]:
tmp.loc[(~tmp.gem_icon.isna()) & (tmp.peptide_HLA_icon == 'KLGGALQAK A0301'),['gem','gem_icon','peptide_HLA','peptide_HLA_icon']] #& (tmp.peptide_HLA != 'KLGGALQAK A0301')

,gem,gem_icon,peptide_HLA,peptide_HLA_icon
0,AAACCTGAGAAACCGC-1-s3,AAACCTGAGAAACCGC-1,KLGGALQAK A0301,KLGGALQAK A0301
45,AAACCTGAGCGTAGTG-3-s3,AAACCTGAGCGTAGTG-3,KLGGALQAK A0301,KLGGALQAK A0301
74,AAACCTGAGGGTCTCC-6-s3,AAACCTGAGGGTCTCC-6,KLGGALQAK A0301,KLGGALQAK A0301
132,AAACCTGCACCCTATC-8-s3,AAACCTGCACCCTATC-8,KLGGALQAK A0301,KLGGALQAK A0301
169,AAACCTGCAGTCAGAG-28-s1,AAACCTGCAGTCAGAG-16,KLGGALQAK A0301,KLGGALQAK A0301
...,...,...,...,...
182701,TTTGTCACACGTCTCT-14-s2,TTTGTCACACGTCTCT-9,KLGGALQAK A0301,KLGGALQAK A0301
182728,TTTGTCACAGTGGGAT-36-s1,TTTGTCACAGTGGGAT-30,KLGGALQAK A0301,KLGGALQAK A0301
182739,TTTGTCACATCTGGTA-30-s2,TTTGTCACATCTGGTA-14,KLGGALQAK A0301,KLGGALQAK A0301
182798,TTTGTCAGTCTCCATC-1-s3,TTTGTCAGTCTCCATC-1,KLGGALQAK A0301,KLGGALQAK A0301


In [136]:
#gem_dups = set(tmp[tmp.duplicated(subset=['gem'], keep=False) & ~tmp.gem_icon.isna()].gem.unique()) - set(tmp[tmp.duplicated(subset=['gem'], keep=False) & (~tmp.gem_icon.isna()) & (tmp.cdr3_TRA == tmp.alpha_cdr3) & (tmp.cdr3_TRB == tmp.beta_cdr3)].gem.unique())

In [137]:
#icon_dups = set(tmp[tmp.duplicated(subset=['Cell barcode'], keep=False) & ~tmp.gem_icon.isna()].gem.unique()) - set(tmp[tmp.duplicated(subset=['Cell barcode'], keep=False) & (~tmp.gem_icon.isna()) & (tmp.cdr3_TRA == tmp.alpha_cdr3) & (tmp.cdr3_TRB == tmp.beta_cdr3)].gem.unique())

In [138]:
#tmp.loc[tmp.gem.isin(gem_dups) & ~tmp.gem.isin(icon_dups) & (tmp.cdr3_TRA != tmp.alpha_cdr3) & (tmp.cdr3_TRB != tmp.beta_cdr3), #tmp.duplicated(subset=['gem'], keep=False) & ~tmp.gem_icon.isna(),
#        ['gem','gem_icon','Cell barcode','clonotype','ct','peptide_HLA','peptide_HLA_icon','cdr3_TRA','alpha_cdr3','cdr3_TRB','beta_cdr3','clonotype','ct']].sort_values(by=['gem','Cell barcode']).head(60)

# Investigating imputations

In [171]:
icon[icon.gem_spl == 'AAACCTGAGCCGATTT']

,Cell barcode,Donor,pMHC,beta_v_gene,beta_j_gene,beta_cdr3,alpha_v_gene,alpha_j_gene,alpha_cdr3,UMI,corrected dex. Sig.,normalized dex. Sig.,gem,gem_spl,sample_id,HLA_cd8,peptide_HLA,HLA_mhc
7583,D2.AAACCTGAGCCGATTT-6,Donor 2,A*1101_IVTDFSVIK_EBNA 3B/EBV,TRBV5-6,TRBJ1-2,CASSSYTGQGGFKNYGYTF,TRAV14/DV4,TRAJ21,CAMREADFNKFYF,148,4.930427,41.044945,AAACCTGAGCCGATTT-6,AAACCTGAGCCGATTT,s2,"[A0201, A0101, B0801]",IVTDFSVIK A1101,A1101


In [172]:
contig[contig.gem_spl == 'AAACCTGAGCCGATTT'] #'AAAGATGTCTCTTATG' 'ACGCCAGAGTACGCCC'

,level_0,level_1,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,...,productive,cdr3,cdr3_nt,reads,umis,raw_clonotype_id,raw_consensus_id,gem,gem_spl,sample_id
187222,s2,78,AAACCTGAGCCGATTT-3-s2,True,AAACCTGAGCCGATTT-3_contig_1,True,499,TRB,None,None,...,False,None,None,2966,2,clonotype34324,None,AAACCTGAGCCGATTT-3,AAACCTGAGCCGATTT,s2
187223,s2,79,AAACCTGAGCCGATTT-3-s2,True,AAACCTGAGCCGATTT-3_contig_2,False,425,TRB,None,None,...,False,None,None,2676,1,clonotype34324,None,AAACCTGAGCCGATTT-3,AAACCTGAGCCGATTT,s2
187224,s2,80,AAACCTGAGCCGATTT-3-s2,True,AAACCTGAGCCGATTT-3_contig_3,True,686,TRB,TRBV5-6,TRBD2,...,True,CASSWSSAGGPSYEQYF,TGTGCCAGCAGCTGGTCCTCGGCGGGGGGACCGAGCTACGAGCAGT...,2373,2,clonotype34324,clonotype3984_consensus_1,AAACCTGAGCCGATTT-3,AAACCTGAGCCGATTT,s2
187225,s2,81,AAACCTGAGCCGATTT-3-s2,True,AAACCTGAGCCGATTT-3_contig_4,True,332,Multi,None,None,...,False,None,None,696,2,clonotype34324,None,AAACCTGAGCCGATTT-3,AAACCTGAGCCGATTT,s2
187226,s2,82,AAACCTGAGCCGATTT-3-s2,True,AAACCTGAGCCGATTT-3_contig_5,True,582,TRA,TRAV14DV4,None,...,True,CAMREADFNKFYF,TGTGCAATGAGAGAGGCAGACTTCAACAAATTTTACTTT,16602,7,clonotype34324,clonotype3984_consensus_2,AAACCTGAGCCGATTT-3,AAACCTGAGCCGATTT,s2


# Any mismatches

In [170]:
tmp.loc[(~tmp.gem_icon.isna()) &
       ~(tmp.ca_match & tmp.cb_match & tmp.va_match & tmp.ja_match & tmp.vb_match & tmp.jb_match),
       ['gem','gem_icon','cdr3_TRA','alpha_cdr3','cdr3_TRB','beta_cdr3',
        'v_gene_TRA','alpha_v_gene','va_match',
        'j_gene_TRA','alpha_j_gene','ja_match',
        'v_gene_TRB','beta_v_gene','vb_match',
        'j_gene_TRB','beta_j_gene','jb_match','clonotype']].sort_values(by=['gem','gem_icon'])#.head(10)

,gem,gem_icon,cdr3_TRA,alpha_cdr3,cdr3_TRB,beta_cdr3,v_gene_TRA,alpha_v_gene,va_match,j_gene_TRA,alpha_j_gene,ja_match,v_gene_TRB,beta_v_gene,vb_match,j_gene_TRB,beta_j_gene,jb_match,clonotype
39,AAACCTGAGCCGATTT-3-s2,AAACCTGAGCCGATTT-6,CAMREADFNKFYF,CAMREADFNKFYF,CASSWSSAGGPSYEQYF,CASSSYTGQGGFKNYGYTF,TRAV14DV4,TRAV14/DV4,True,TRAJ21,TRAJ21,True,TRBV5-6,TRBV5-6,True,TRBJ2-7,TRBJ1-2,False,clonotype34324
61,AAACCTGAGGCCCGTT-22-s2,AAACCTGAGGCCCGTT-17,NaN,CAETQDSNYQLIW,CASSTSTGTGYGYTF,CASSTSTGTGYGYTF,NaN,TRAV5,False,NaN,TRAJ33,False,TRBV4-1,TRBV4-1,True,TRBJ1-2,TRBJ1-2,True,clonotype30394
75,AAACCTGAGGTGCACA-13-s1,AAACCTGAGGTGCACA-31,CAVGDNFNKFYF,CAARVRGFGNVLHC,CASSLYSATGELFF,CASSLYSATGELFF,TRAV8-3,TRAV13-1,False,TRAJ21,TRAJ35,False,TRBV28,TRBV28,True,TRBJ2-2,TRBJ2-2,True,clonotype2
114,AAACCTGCAATAGCGG-9-s1,AAACCTGCAATAGCGG-12,CAGHTGNQFYF,CAGHTGNQFYF,NaN,CASSWGGGSHYGYTF,TRAV35,TRAV35,True,TRAJ49,TRAJ49,True,NaN,TRBV11-2,False,NaN,TRBJ1-2,False,clonotype46
129,AAACCTGCACCAGTTA-4-s1,AAACCTGCACCAGTTA-32,CAVGDNFNKFYF,CAARVRGFGNVLHC,CASSLYSATGELFF,CASSLYSATGELFF,TRAV8-3,TRAV13-1,False,TRAJ21,TRAJ35,False,TRBV28,TRBV28,True,TRBJ2-2,TRBJ2-2,True,clonotype2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182661,TTTGTCACAAGCCATT-20-s1,TTTGTCACAAGCCATT-22,NaN,CAGHTGNQFYF,CASSWGGGSHYGYTF,CASSWGGGSHYGYTF,NaN,TRAV35,False,NaN,TRAJ49,False,TRBV11-2,TRBV11-2,True,TRBJ1-2,TRBJ1-2,True,clonotype3
182701,TTTGTCACACGTCTCT-14-s2,TTTGTCACACGTCTCT-9,CAVIRGNTGNQFYF,CILMRGAQKLVF,CASSLGGGGYNEQFF,CASSLGGGGYNEQFF,TRAV41,TRAV26-2,False,TRAJ49,TRAJ54,False,TRBV27,TRBV27,True,TRBJ2-1,TRBJ2-1,True,clonotype32530
182708,TTTGTCACAGATCTGT-9-s2,TTTGTCACAGATCTGT-12,CAASETSYDKVIF,CAASETSYDKVIF,NaN,CASSFSGNTGELFF,TRAV13-1,TRAV13-1,True,TRAJ50,TRAJ50,True,NaN,TRBV12-3,False,NaN,TRBJ2-2,False,clonotype30474
182853,TTTGTCATCACCTTAT-38-s2,TTTGTCATCACCTTAT-34,CADSGGGADGLTF,CADSGGGADGLTF,NaN,CASSLRDGSEAFF,TRAV13-2,TRAV13-2,True,TRAJ45,TRAJ45,True,NaN,TRBV5-6,False,NaN,TRBJ1-1,False,clonotype30467


In [101]:
df.gem.nunique()

181913

## Complete mismatches

In [30]:
df.loc[(~df.gem_icon.isna()) &
       ~(df.v_alpha_match & df.j_alpha_match & df.v_beta_match & df.j_beta_match),
       ['gem','gem_icon','cdr3_TRA','alpha_cdr3','cdr3_TRB','beta_cdr3',
        'v_gene_TRA','alpha_v_gene','v_alpha_match',
        'j_gene_TRA','alpha_j_gene','j_alpha_match',
        'v_gene_TRB','beta_v_gene','v_beta_match',
        'j_gene_TRB','beta_j_gene','j_beta_match','clonotype']].sort_values(by=['gem','gem_icon'])#.head(10)

,gem,gem_icon,cdr3_TRA,alpha_cdr3,cdr3_TRB,beta_cdr3,v_gene_TRA,alpha_v_gene,v_alpha_match,j_gene_TRA,alpha_j_gene,j_alpha_match,v_gene_TRB,beta_v_gene,v_beta_match,j_gene_TRB,beta_j_gene,j_beta_match,clonotype
39,AAACCTGAGCCGATTT-3-s2,AAACCTGAGCCGATTT-6,CAMREADFNKFYF,CAMREADFNKFYF,CASSWSSAGGPSYEQYF,CASSSYTGQGGFKNYGYTF,TRAV14DV4,TRAV14/DV4,True,TRAJ21,TRAJ21,True,TRBV5-6,TRBV5-6,True,TRBJ2-7,TRBJ1-2,False,clonotype34324
61,AAACCTGAGGCCCGTT-22-s2,AAACCTGAGGCCCGTT-17,NaN,CAETQDSNYQLIW,CASSTSTGTGYGYTF,CASSTSTGTGYGYTF,NaN,TRAV5,False,NaN,TRAJ33,False,TRBV4-1,TRBV4-1,True,TRBJ1-2,TRBJ1-2,True,clonotype30394
62,AAACCTGAGGCCCGTT-7-s2,AAACCTGAGGCCCGTT-17,NaN,CAETQDSNYQLIW,CASSLRDGSEAFF,CASSTSTGTGYGYTF,NaN,TRAV5,False,NaN,TRAJ33,False,TRBV5-6,TRBV4-1,False,TRBJ1-1,TRBJ1-2,False,clonotype30344
75,AAACCTGAGGTGCACA-13-s1,AAACCTGAGGTGCACA-31,CAVGDNFNKFYF,CAARVRGFGNVLHC,CASSLYSATGELFF,CASSLYSATGELFF,TRAV8-3,TRAV13-1,False,TRAJ21,TRAJ35,False,TRBV28,TRBV28,True,TRBJ2-2,TRBJ2-2,True,clonotype2
114,AAACCTGCAATAGCGG-9-s1,AAACCTGCAATAGCGG-12,CAGHTGNQFYF,CAGHTGNQFYF,NaN,CASSWGGGSHYGYTF,TRAV35,TRAV35,True,TRAJ49,TRAJ49,True,NaN,TRBV11-2,False,NaN,TRBJ1-2,False,clonotype46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181709,TTTGTCACATGCCACG-20-s2,TTTGTCACATGCCACG-22,CAMKSTNAGKSTF,CAASETSYDKVIF,CASSLGVAGTNTGELFF,CASSFSGNTGELFF,TRAV12-3,TRAV13-1,False,TRAJ27,TRAJ50,False,TRBV11-2,TRBV12-3,False,TRBJ2-2,TRBJ2-2,True,clonotype30345
181817,TTTGTCATCAAAGTAG-27-s2,TTTGTCATCAAAGTAG-12,CASNYGGSQGNLIF,CAPWDFNKFYF,CASSMRSSYEQYF,CASSLQGGTGELFF,TRAV27,TRAV13-1,False,TRAJ42,TRAJ21,False,TRBV19,TRBV7-3,False,TRBJ2-7,TRBJ2-2,False,clonotype31547
181820,TTTGTCATCACCTTAT-38-s2,TTTGTCATCACCTTAT-34,CADSGGGADGLTF,CADSGGGADGLTF,NaN,CASSLRDGSEAFF,TRAV13-2,TRAV13-2,True,TRAJ45,TRAJ45,True,NaN,TRBV5-6,False,NaN,TRBJ1-1,False,clonotype30467
181826,TTTGTCATCACTTCAT-22-s2,TTTGTCATCACTTCAT-17,NaN,CAETHSGGSNYKLTF,CSVGSGDHGEQFF,CSVGSGDHGEQFF,NaN,TRAV13-2,False,NaN,TRAJ53,False,TRBV29-1,TRBV29-1,True,TRBJ2-1,TRBJ2-1,True,clonotype30433


In [31]:
smpl = df.groupby('sample_id')

In [36]:
smpl1 = smpl.get_group('s1')

In [43]:
nans = smpl1[['v_gene_TRA','j_gene_TRA','v_gene_TRB','j_gene_TRB']].isna().any(axis=1)

In [56]:
lol = smpl1.loc[nans & ~smpl1.gem_icon.isna(),
          ['gem','gem_icon','cdr3_TRA','alpha_cdr3','cdr3_TRB','beta_cdr3','clonotype','ct']].sort_values(by=['gem','gem_icon'])

In [65]:
loa = lol.dropna(subset=['cdr3_TRA'])

In [66]:
loa

,gem,gem_icon,cdr3_TRA,alpha_cdr3,cdr3_TRB,beta_cdr3,clonotype,ct
114,AAACCTGCAATAGCGG-9-s1,AAACCTGCAATAGCGG-12,CAGHTGNQFYF,CAGHTGNQFYF,NaN,CASSWGGGSHYGYTF,clonotype46,46
2574,AACACGTGTCAGAATA-25-s1,AACACGTGTCAGAATA-26,CAGHTGNQFYF,CAGHTGNQFYF,NaN,CASSWGGGSHYGYTF,clonotype46,46
3935,AACTCAGAGTGTCCAT-12-s1,AACTCAGAGTGTCCAT-3,CAGHTGNQFYF,CAGHTGNQFYF,NaN,CASSWGGGSHYGYTF,clonotype46,46
4820,AACTCTTGTAGCTAAA-21-s1,AACTCTTGTAGCTAAA-18,CALHPHTGANSKLTF,CALHPHTGANSKLTF,NaN,CASSIGVYGYTF,clonotype2064,2064
4898,AACTCTTGTTCGAATC-5-s1,AACTCTTGTTCGAATC-1,CAGHTGNQFYF,CAGHTGNQFYF,NaN,CASSWGGGSHYGYTF,clonotype46,46
...,...,...,...,...,...,...,...,...
167125,TGCTACCTCCTGTACC-3-s1,TGCTACCTCCTGTACC-6,CAVGDNFNKFYF,CAVGDNFNKFYF,NaN,CASSLYSATGELFF,clonotype83,83
173670,TTAGTTCAGTTCGCAT-5-s1,TTAGTTCAGTTCGCAT-1,CAADNYGGSQGNLIF,CAADNYGGSQGNLIF,NaN,CASSIYGGAGDGYTF,clonotype328,328
176286,TTCTCCTAGAGTGACC-17-s1,TTCTCCTAGAGTGACC-5,CAFMTNAGGTSYGKLTF,CAFMTNAGGTSYGKLTF,NaN,CASSQGAYGYTF,clonotype1276,1276
176405,TTCTCCTCAATCTGCA-3-s1,TTCTCCTCAATCTGCA-6,CAGHTGNQFYF,CAGHTGNQFYF,NaN,CASSWGGGSHYGYTF,clonotype46,46


## Checking imputations of missing chain

In [197]:
total_imps = 0
wrong_imps = 0
for smpl,grp in tmp.dropna(subset=['gem_icon']).groupby('sample_id'): # df
    print(smpl)
    clono = get_clono() #smpl
    nans = grp[['cdr3_TRA','cdr3_TRB']].isna().any(axis=1) # Test also for imputations when there actually was a chain!
    lol = grp[nans]
    total_imps += len(lol)
    for c_query, c_missing in zip(['TRA','TRB'], ['TRB','TRA']):
        print(c_query)
        lo = lol.dropna(subset=[f'cdr3_{c_query}'])
        reliable = list()
        for cdr3 in lo[f'cdr3_{c_query}'].unique():
            dups = clono.cdr3s_aa.str.contains(cdr3)
            trxs = sum(dups & clono.cdr3s_aa.str.contains(c_missing))
            if (sum(dups) <= 2) & (trxs == 1):
                reliable.append(cdr3)
        wrong_imps += sum(~lo[f'cdr3_{c_query}'].isin(reliable))
        print(reliable)

s1
TRA
['CAVLGYNTDKLIF', 'CAYRISLNNNDMRF', 'CAASPTGMDSNYQLIW', 'CAVGRGSYIPTF', 'CAVPTRSGANNLFF']
TRB
['CASTPDRGLTEAFF', 'CASSVGGVSPLHF', 'CASRMGDINTGELFF', 'CASNPSSTDTQYF', 'CASSGTSVHEQYF']
s2
TRA
['CAGAGHGGSQGNLIF', 'CAMSGHGGSQGNLIF', 'CATNGDSGYSTLTF', 'CAVNTPPGDSNYQLIW', 'CIVREVGDKLIF', 'CAADDKIIF', 'CADEGEGGGADGLTF', 'CVVNLLNYQLIW', 'CATPESSASKIIF', 'CAAEETSYDKVIF', 'CAPWDFNKFYF', 'CALSEANFGNEKLTF', 'CAMREEGMDSNYQLIW', 'CAEEGGGGSQGNLIF', 'CVPYSGGGADGLTF', 'CAPSAGGGSQGNLIF', 'CAVSYNQGGKLIF']
TRB
['CSAGNSGGYNEQFF', 'CASSLGQNTEAFF', 'CASSQEVLGGPYGYTF', 'CASSSLGTAKSPLHF', 'CASSQRSSYEQYF', 'CASSRLSNQPQHF', 'CASSIRSGNEQFF', 'CASSPHRGLEGYTF', 'CASTSVQRQGGDASYEQYF', 'CATSDSQLTSGGETGELFF', 'CASSLEMAGGRETQYF', 'CASSIRSDEKLFF', 'CASNPTDGGETQYF', 'CASSYNPRTEVSNQPQHF', 'CASSFSGANYGYTF', 'CASSIRSSDEQFF', 'CTTSSGEADTQYF', 'CASMELAGLNEQFF', 'CASSLRASYEQYF', 'CASSIRSSNEQYF', 'CASGPDVDNSPLHF', 'CASSVRSAYEQYF', 'CASRYPGGLDEQYF', 'CAASSRSTDTQYF', 'CSARPQPVGNTIYF', 'CASSIHGLETQYF']
s3
TRA
[]
TRB
[]
s4
T

In [144]:
wrong_imps # tmp

2330

In [145]:
total_imps # tmp

2446

In [119]:
wrong_imps # old

2330

In [120]:
total_imps

2446

In [27]:
df.loc[(df.v_alpha_match == True) & 
   (df.j_alpha_match == True) & 
   (df.v_beta_match == True) & 
   (df.j_beta_match == True),
       ['gem','gem_icon','cdr3_TRA','alpha_cdr3',
        'cdr3_TRB','beta_cdr3','clonotype','ct']].dropna(subset=['gem_icon','cdr3_TRA','cdr3_TRB']).shape#.sort_values(by=['gem','gem_icon'])

(33349, 8)

## Total mismatches of fully annotated chains

In [245]:
tmp.loc[(tmp.va_match == False) & 
   (tmp.ja_match == False) & 
   (tmp.vb_match == False) & 
   (tmp.jb_match == False),
       ['gem','gem_icon','cdr3_TRA','alpha_cdr3',
        'cdr3_TRB','beta_cdr3','clonotype','ct']].dropna(subset=['gem_icon','cdr3_TRA','cdr3_TRB']).sort_values(by=['gem','gem_icon'])

,gem,gem_icon,cdr3_TRA,alpha_cdr3,cdr3_TRB,beta_cdr3,clonotype,ct
6875,AAGGCAGAGAGGGATA-33-s1,AAGGCAGAGAGGGATA-12,CAVGDSGGGADGLTF,CAGHTGNQFYF,CASSLNPNTEAFF,CASSWGGGSHYGYTF,clonotype2410,2410
6908,AAGGCAGAGGACTGGT-12-s2,AAGGCAGAGGACTGGT-25,CAALGAGGTSYGKLTF,CAASETSYDKVIF,CASSQDRPQLAGEKTQYF,CASSFSGNTGELFF,clonotype30694,30694
8158,AATCCAGCATGGATGG-12-s3,AATCCAGCATGGATGG-8,CAFIGDAGGTSYGKLTF,CAMRESMNYGGSQGNLIF,CASSQDHRMGGHEKLFF,CASSPGQWTKNIQYF,clonotype50996,50996
11473,ACATCAGAGCTGCGAA-3-s1,ACATCAGAGCTGCGAA-6,CAVGDNFNKFYF,CAARVRGFGNVLHC,CASSVRSSMNTEAFF,CASSLYSATGELFF,clonotype3166,3166
14755,ACGATACGTTCGCGAC-39-s1,ACGATACGTTCGCGAC-18,CAVRGGYQKVTF,CAGGGSQGNLIF,CSARDVEGGDQYGYTF,CASSIRSSYEQYF,clonotype3674,3674
15060,ACGATGTCATGCGCAC-21-s1,ACGATGTCATGCGCAC-18,CAAINNAGNMLTF,CAFMKHENSGTYKYIF,CASSISLAAEQFF,CATSDSGRVNTEAFF,clonotype3722,3722
15488,ACGCAGCGTCAAAGAT-23-s2,ACGCAGCGTCAAAGAT-25,CAGRTTNTGTASKLTF,CAFMKGAGGTSYGKLTF,CASSLGQYNSPLHF,CASSIGVHGYTF,clonotype35707,35707
18679,ACTATCTTCTTTAGGG-33-s2,ACTATCTTCTTTAGGG-26,CIVRVPESHTL,CATDASGGSNYKLTF,CASSVGTGDHQPQHF,CSVGTGDWGEQYF,clonotype32345,32345
21177,ACTTGTTGTCAACTGT-38-s1,ACTTGTTGTCAACTGT-18,CALSEASGTYKYIF,CAARVRGFGNVLHC,CSAPSGEGRDTQYF,CASSLYSATGELFF,clonotype4651,4651
25466,AGCCTAAGTGAGGGAG-29-s2,AGCCTAAGTGAGGGAG-25,CALKLGNAGNMLTF,CAGGGSQGNLIF,CASSVNPNPIRDRDNEKLFF,CASSIRSSYEQYF,clonotype30744,30744


In [258]:
tmp.loc[~((tmp.cdr3_TRA == tmp.alpha_cdr3) & (tmp.cdr3_TRB == tmp.beta_cdr3)),
       ['gem','gem_icon','cdr3_TRA','alpha_cdr3',
        'cdr3_TRB','beta_cdr3','clonotype','ct']].dropna(subset=['gem_icon']).sort_values(by=['gem','gem_icon'])

,gem,gem_icon,cdr3_TRA,alpha_cdr3,cdr3_TRB,beta_cdr3,clonotype,ct
39,AAACCTGAGCCGATTT-3-s2,AAACCTGAGCCGATTT-6,CAMREADFNKFYF,CAMREADFNKFYF,CASSWSSAGGPSYEQYF,CASSSYTGQGGFKNYGYTF,clonotype34324,34324
61,AAACCTGAGGCCCGTT-22-s2,AAACCTGAGGCCCGTT-17,NaN,CAETQDSNYQLIW,CASSTSTGTGYGYTF,CASSTSTGTGYGYTF,clonotype30394,11681
75,AAACCTGAGGTGCACA-13-s1,AAACCTGAGGTGCACA-31,CAVGDNFNKFYF,CAARVRGFGNVLHC,CASSLYSATGELFF,CASSLYSATGELFF,clonotype2,2
114,AAACCTGCAATAGCGG-9-s1,AAACCTGCAATAGCGG-12,CAGHTGNQFYF,CAGHTGNQFYF,NaN,CASSWGGGSHYGYTF,clonotype46,46
129,AAACCTGCACCAGTTA-4-s1,AAACCTGCACCAGTTA-32,CAVGDNFNKFYF,CAARVRGFGNVLHC,CASSLYSATGELFF,CASSLYSATGELFF,clonotype2,2
...,...,...,...,...,...,...,...,...
182661,TTTGTCACAAGCCATT-20-s1,TTTGTCACAAGCCATT-22,NaN,CAGHTGNQFYF,CASSWGGGSHYGYTF,CASSWGGGSHYGYTF,clonotype3,3
182701,TTTGTCACACGTCTCT-14-s2,TTTGTCACACGTCTCT-9,CAVIRGNTGNQFYF,CILMRGAQKLVF,CASSLGGGGYNEQFF,CASSLGGGGYNEQFF,clonotype32530,32530
182708,TTTGTCACAGATCTGT-9-s2,TTTGTCACAGATCTGT-12,CAASETSYDKVIF,CAASETSYDKVIF,NaN,CASSFSGNTGELFF,clonotype30474,30474
182853,TTTGTCATCACCTTAT-38-s2,TTTGTCATCACCTTAT-34,CADSGGGADGLTF,CADSGGGADGLTF,NaN,CASSLRDGSEAFF,clonotype30467,13140


In [257]:
tmp.loc[((tmp.cdr3_TRA != tmp.alpha_cdr3) | (tmp.cdr3_TRB != tmp.beta_cdr3)),
       ['gem','gem_icon','cdr3_TRA','alpha_cdr3',
        'cdr3_TRB','beta_cdr3','clonotype','ct']].dropna(subset=['gem_icon']).dropna(subset=['cdr3_TRA','cdr3_TRB']).sort_values(by=['gem','gem_icon'])

,gem,gem_icon,cdr3_TRA,alpha_cdr3,cdr3_TRB,beta_cdr3,clonotype,ct
39,AAACCTGAGCCGATTT-3-s2,AAACCTGAGCCGATTT-6,CAMREADFNKFYF,CAMREADFNKFYF,CASSWSSAGGPSYEQYF,CASSSYTGQGGFKNYGYTF,clonotype34324,34324
75,AAACCTGAGGTGCACA-13-s1,AAACCTGAGGTGCACA-31,CAVGDNFNKFYF,CAARVRGFGNVLHC,CASSLYSATGELFF,CASSLYSATGELFF,clonotype2,2
129,AAACCTGCACCAGTTA-4-s1,AAACCTGCACCAGTTA-32,CAVGDNFNKFYF,CAARVRGFGNVLHC,CASSLYSATGELFF,CASSLYSATGELFF,clonotype2,2
139,AAACCTGCACTTAAGC-16-s2,AAACCTGCACTTAAGC-21,CAEMEGGSYIPTF,CAESTSGGKLIF,CSVGSRGTNEKLFF,CSVGSRGTNEKLFF,clonotype34329,34329
159,AAACCTGCAGGTGGAT-23-s2,AAACCTGCAGGTGGAT-20,CAGGGSQGNLIF,CAGGGSQGNLIF,CASSMGDRSMGNIQYF,CASSIRSSYEQYF,clonotype30361,30361
...,...,...,...,...,...,...,...,...
182048,TTTGCGCGTTAAGAAC-33-s2,TTTGCGCGTTAAGAAC-10,CAASETSYDKVIF,CAASETSYDKVIF,CASRGSVYEQYF,CASSFSGNTGELFF,clonotype50914,50914
182538,TTTGGTTTCTTCGGTC-12-s2,TTTGGTTTCTTCGGTC-3,CAVGDNFNKFYF,CAAISGAGSYQLTF,CASSYGTGENTEAFF,CASSYGTGENTEAFF,clonotype50956,50956
182583,TTTGTCAAGCCCTAAT-12-s1,TTTGTCAAGCCCTAAT-3,CAVGDNFNKFYF,CAARVRGFGNVLHC,CASSLYSATGELFF,CASSLYSATGELFF,clonotype2,2
182589,TTTGTCAAGCGTTGCC-31-s2,TTTGTCAAGCGTTGCC-38,CAGPGDGGSQGNLIF,CAGPGDGGSQGNLIF,CASSYSLQGGLDEQYF,CASSIRSSYEQYF,clonotype50962,50962


In [256]:
tmp.loc[(tmp.cdr3_TRA.isna() | tmp.cdr3_TRB.isna()),
       ['gem','gem_icon','cdr3_TRA','alpha_cdr3',
        'cdr3_TRB','beta_cdr3','clonotype','ct']].dropna(subset=['gem_icon']).sort_values(by=['gem','gem_icon']) #,'cdr3_TRA','cdr3_TRB'

,gem,gem_icon,cdr3_TRA,alpha_cdr3,cdr3_TRB,beta_cdr3,clonotype,ct
61,AAACCTGAGGCCCGTT-22-s2,AAACCTGAGGCCCGTT-17,NaN,CAETQDSNYQLIW,CASSTSTGTGYGYTF,CASSTSTGTGYGYTF,clonotype30394,11681
114,AAACCTGCAATAGCGG-9-s1,AAACCTGCAATAGCGG-12,CAGHTGNQFYF,CAGHTGNQFYF,NaN,CASSWGGGSHYGYTF,clonotype46,46
191,AAACCTGGTACCATCA-35-s1,AAACCTGGTACCATCA-40,NaN,CAGHTGNQFYF,CASSWGGGSHYGYTF,CASSWGGGSHYGYTF,clonotype3,3
210,AAACCTGGTCATACTG-25-s2,AAACCTGGTCATACTG-27,NaN,CAASETSYDKVIF,CASSFSGNTGELFF,CASSFSGNTGELFF,clonotype30346,1275
366,AAACCTGTCTGTTTGT-7-s2,AAACCTGTCTGTTTGT-22,NaN,CAGAQGGGSQGNLIF,CASSIRAAYEQYF,CASSIRAAYEQYF,clonotype30980,30980
...,...,...,...,...,...,...,...,...
182561,TTTGTCAAGAGTACAT-37-s1,TTTGTCAAGAGTACAT-11,NaN,CAGHTGNQFYF,CASSWGGGSHYGYTF,CASSWGGGSHYGYTF,clonotype3,3
182661,TTTGTCACAAGCCATT-20-s1,TTTGTCACAAGCCATT-22,NaN,CAGHTGNQFYF,CASSWGGGSHYGYTF,CASSWGGGSHYGYTF,clonotype3,3
182708,TTTGTCACAGATCTGT-9-s2,TTTGTCACAGATCTGT-12,CAASETSYDKVIF,CAASETSYDKVIF,NaN,CASSFSGNTGELFF,clonotype30474,30474
182853,TTTGTCATCACCTTAT-38-s2,TTTGTCATCACCTTAT-34,CADSGGGADGLTF,CADSGGGADGLTF,NaN,CASSLRDGSEAFF,clonotype30467,13140


In [254]:
1854+1537

3391

In [108]:
df.loc[(df.v_alpha_match == False) & 
   (df.j_alpha_match == False) & 
   (df.v_beta_match == False) & 
   (df.j_beta_match == False),
       ['gem','gem_icon','cdr3_TRA','alpha_cdr3',
        'cdr3_TRB','beta_cdr3','clonotype','ct']].dropna(subset=['gem_icon','cdr3_TRA','cdr3_TRB']).sort_values(by=['gem','gem_icon'])

,gem,gem_icon,cdr3_TRA,alpha_cdr3,cdr3_TRB,beta_cdr3,clonotype,ct
283,AAACCTGTCACGATGT-11-s1,AAACCTGTCACGATGT-20,CALSEADDSWGKLQF,CAGHTGNQFYF,CASSLYGNLGTGELFF,CASSWGGGSHYGYTF,clonotype560,560
301,AAACCTGTCCAAGTAC-34-s1,AAACCTGTCCAAGTAC-16,CALKLIKAAGNKLTF,CAVSLGGGSQGNLIF,CAWTGTGKIGWDSPLHF,CASSGRSTDTQYF,clonotype10,10
340,AAACCTGTCTAACTTC-38-s2,AAACCTGTCTAACTTC-13,CAASMGGGSQGNLIF,CAEMEGGSYIPTF,CASSIRSTDTQYF,CASSQRPSEVGELFF,clonotype30494,30494
341,AAACCTGTCTAACTTC-6-s2,AAACCTGTCTAACTTC-34,CAEMEGGSYIPTF,CAASMGGGSQGNLIF,CASSQRPSEVGELFF,CASSIRSTDTQYF,clonotype30347,30347
422,AAACGGGAGCGATGAC-40-s2,AAACGGGAGCGATGAC-14,CAVRVTSLTGGGNKLTF,CADSGGGADGLTF,CASSFSGQGLTNEKLFF,CASSLRDGSEAFF,clonotype31574,31574
...,...,...,...,...,...,...,...,...
181427,TTTGGTTTCATCTGCC-2-s3,TTTGGTTTCATCTGCC-3,CAVRSSGSARQLTF,CIPERLGNNNAGNMLTF,CASSLEPSYEQYF,CASSAGSRAYNEQFF,clonotype51002,51002
181433,TTTGGTTTCCACGTTC-26-s2,TTTGGTTTCCACGTTC-10,CALDGPGNTDKLIF,CAEMEGGSYIPTF,CASSPGVYGYTF,CASSQRPSEVGELFF,clonotype30911,30911
181531,TTTGTCAAGATACACA-34-s2,TTTGTCAAGATACACA-18,CALLEMVIKAAGNKLTF,CAASETSYDKVIF,CASSTDRAAQPQHF,CASSFSGNTGELFF,clonotype30417,30417
181617,TTTGTCAAGTTGTCGT-34-s2,TTTGTCAAGTTGTCGT-32,CAVNPWGNDYKLSF,CADSGGGADGLTF,CASSTAGGSYEQYF,CASSLRDGSEAFF,clonotype31546,31546


# contig test

In [29]:
tmp.loc[tmp.cdr3_TRA.isna() & (tmp.cdr3_TRB == tmp.beta_cdr3)] #& (tmp.cdr3_TRA != tmp.alpha_cdr3) 

,gem,clonotype,num_clonotype,ct,genes_TRA,genes_TRB,genes_lst_TRA,genes_lst_TRB,length_TRA,cdr3_TRA,...,peptide_HLA_icon,HLA_mhc_icon,ca_match,cb_match,va_match,ja_match,vb_match,jb_match,peptide_HLA_match,ct_icon
61,AAACCTGAGGCCCGTT-22-s2,clonotype30394,30394,11681,NaN,TRBV4-1;TRBD1;TRBJ1-2;TRBC1,NaN,['TRBV4-1;TRBD1;TRBJ1-2;TRBC1'],NaN,NaN,...,RAKFKQLL B0801,B0801,False,True,False,False,True,True,True,12.0
191,AAACCTGGTACCATCA-35-s1,clonotype3,3,3,NaN,TRBV11-2;TRBD1;TRBJ1-2;TRBC1,NaN,['TRBV11-2;TRBD1;TRBJ1-2;TRBC1'],NaN,NaN,...,IVTDFSVIK A1101,A1101,False,True,False,False,True,True,True,3.0
210,AAACCTGGTCATACTG-25-s2,clonotype30346,30346,1275,NaN,TRBV12-3;None;TRBJ2-2;TRBC2,NaN,['TRBV12-3;None;TRBJ2-2;TRBC2'],NaN,NaN,...,RAKFKQLL B0801,B0801,False,True,False,False,True,True,True,1.0
366,AAACCTGTCTGTTTGT-7-s2,clonotype30980,30980,30980,NaN,TRBV19;TRBD1;TRBJ2-7;TRBC2,NaN,['TRBV19;TRBD1;TRBJ2-7;TRBC2' 'TRBV3-1;TRBD1;T...,NaN,NaN,...,GILGFVFTL A0201,A0201,False,True,False,False,True,True,True,63.0
479,AAACGGGAGTCGTACT-17-s1,clonotype3,3,3,NaN,TRBV11-2;TRBD1;TRBJ1-2;TRBC1,NaN,['TRBV11-2;TRBD1;TRBJ1-2;TRBC1'],NaN,NaN,...,IVTDFSVIK A1101,A1101,False,True,False,False,True,True,True,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182451,TTTGGTTTCAGCGATT-13-s2,clonotype30344,30344,1872,NaN,TRBV5-6;TRBD2;TRBJ1-1;TRBC1,NaN,['TRBV5-6;TRBD2;TRBJ1-1;TRBC1'],NaN,NaN,...,RAKFKQLL B0801,B0801,False,True,False,False,True,True,True,2.0
182541,TTTGGTTTCTTGTCAT-12-s1,clonotype3,3,3,NaN,TRBV11-2;TRBD1;TRBJ1-2;TRBC1,NaN,['TRBV11-2;TRBD1;TRBJ1-2;TRBC1'],NaN,NaN,...,IVTDFSVIK A1101,A1101,False,True,False,False,True,True,True,3.0
182561,TTTGTCAAGAGTACAT-37-s1,clonotype3,3,3,NaN,TRBV11-2;TRBD1;TRBJ1-2;TRBC1,NaN,['TRBV11-2;TRBD1;TRBJ1-2;TRBC1'],NaN,NaN,...,IVTDFSVIK A1101,A1101,False,True,False,False,True,True,True,3.0
182661,TTTGTCACAAGCCATT-20-s1,clonotype3,3,3,NaN,TRBV11-2;TRBD1;TRBJ1-2;TRBC1,NaN,['TRBV11-2;TRBD1;TRBJ1-2;TRBC1'],NaN,NaN,...,IVTDFSVIK A1101,A1101,False,True,False,False,True,True,True,3.0


In [30]:
contig[contig.gem_spl == 'AAACCTGAGGCCCGTT']

,level_0,level_1,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,...,productive,cdr3,cdr3_nt,reads,umis,raw_clonotype_id,raw_consensus_id,gem,gem_spl,sample_id
187253,s2,109,AAACCTGAGGCCCGTT-22-s2,True,AAACCTGAGGCCCGTT-22_contig_1,True,662,TRB,TRBV4-1,TRBD1,...,True,CASSTSTGTGYGYTF,TGCGCCAGCAGCACTTCCACCGGGACAGGCTATGGCTACACCTTC,29034,12,clonotype30394,clonotype54_consensus_1,AAACCTGAGGCCCGTT-22,AAACCTGAGGCCCGTT,s2
187254,s2,110,AAACCTGAGGCCCGTT-22-s2,True,AAACCTGAGGCCCGTT-22_contig_2,True,429,TRB,None,None,...,False,None,None,6841,3,clonotype30394,None,AAACCTGAGGCCCGTT-22,AAACCTGAGGCCCGTT,s2
187255,s2,111,AAACCTGAGGCCCGTT-7-s2,True,AAACCTGAGGCCCGTT-7_contig_1,True,850,TRB,TRBV5-6,TRBD2,...,True,CASSLRDGSEAFF,TGTGCCAGCAGCCTACGGGACGGCTCTGAAGCTTTCTTT,13173,7,clonotype30344,clonotype4_consensus_1,AAACCTGAGGCCCGTT-7,AAACCTGAGGCCCGTT,s2
187256,s2,112,AAACCTGAGGCCCGTT-7-s2,True,AAACCTGAGGCCCGTT-7_contig_2,True,468,TRB,None,None,...,False,None,None,6380,4,clonotype30344,None,AAACCTGAGGCCCGTT-7,AAACCTGAGGCCCGTT,s2


In [31]:
icon[icon.gem_spl == 'AAACCTGAGGCCCGTT']

,Cell barcode,Donor,pMHC,beta_v_gene,beta_j_gene,beta_cdr3,alpha_v_gene,alpha_j_gene,alpha_cdr3,UMI,corrected dex. Sig.,normalized dex. Sig.,gem,gem_spl,sample_id,HLA_cd8,peptide_HLA,HLA_mhc
7586,D2.AAACCTGAGGCCCGTT-17,Donor 2,B*0801_RAKFKQLL_BZLF1/EBV,TRBV4-1,TRBJ1-2,CASSTSTGTGYGYTF,TRAV5,TRAJ33,CAETQDSNYQLIW,79,2.571096,1.392421,AAACCTGAGGCCCGTT-17,AAACCTGAGGCCCGTT,s2,"[A0201, A0101, B0801]",RAKFKQLL B0801,B0801


In [32]:
tcr[tcr.gem.str.contains('AAACCTGAGGCCCGTT')]

,gem,clonotype,num_clonotype,ct,genes_TRA,genes_TRB,genes_lst_TRA,genes_lst_TRB,length_TRA,cdr3_TRA,...,v_gene_TRB,j_gene_TRB,tcr_category,no_filtration,exclude_single-chain_TCRs,exclude_ambiguous_and_single-chain_TCRs,exclude_ambiguous_TCRs,label,cell_flag,cell_high_confidence
293232,AAACCTGAGGCCCGTT-22-s2,clonotype30394,30394,11681,NaN,TRBV4-1;TRBD1;TRBJ1-2;TRBC1,NaN,['TRBV4-1;TRBD1;TRBJ1-2;TRBC1'],NaN,NaN,...,TRBV4-1,TRBJ1-2,missing chain,True,False,False,False,1.0,False,False
293233,AAACCTGAGGCCCGTT-7-s2,clonotype30344,30344,1872,NaN,TRBV5-6;TRBD2;TRBJ1-1;TRBC1,NaN,['TRBV5-6;TRBD2;TRBJ1-1;TRBC1'],NaN,NaN,...,TRBV5-6,TRBJ1-1,missing chain,True,False,False,False,1.0,False,False


In [34]:
tmp.loc[tmp.gem_spl == 'AAACCTGAGGCCCGTT', ['gem','gem_icon','cdr3_TRA','alpha_cdr3','cdr3_TRB','beta_cdr3', 'clonotype','ct','peptide_HLA','peptide_HLA_icon']]

,gem,gem_icon,cdr3_TRA,alpha_cdr3,cdr3_TRB,beta_cdr3,clonotype,ct,peptide_HLA,peptide_HLA_icon
62,AAACCTGAGGCCCGTT-7-s2,NaN,NaN,CAETQDSNYQLIW,CASSLRDGSEAFF,CASSTSTGTGYGYTF,clonotype30344,1872,RAKFKQLL B0801,RAKFKQLL B0801
61,AAACCTGAGGCCCGTT-22-s2,AAACCTGAGGCCCGTT-17,NaN,CAETQDSNYQLIW,CASSTSTGTGYGYTF,CASSTSTGTGYGYTF,clonotype30394,11681,RAKFKQLL B0801,RAKFKQLL B0801


In [226]:
contig[(contig.raw_clonotype_id == 'clonotype2') & (contig.chain == 'TRA')].groupby('cdr3').umis.median()

cdr3
CAANERAQKLVF            1.0
CAARVRGFGNVLHC          4.0
CAASGRTGGFKTIF          1.0
CAASSAGAGSYQLTF         1.0
CADSGGGADGLTF           1.0
CAEGKGGGKLIF            1.0
CAENPNAGNMLTF           1.0
CAESEGPTYTDKLIF         1.0
CAFMKGAGGTSYGKLTF       1.0
CAFMSNAGKSTF            1.0
CAFMTNAGGTSYGKLTF       1.0
CAGHTGNQFYF             1.0
CAGKDGSAGGGADGLTF       1.0
CALLTGGFKTIF            1.0
CALSEFEGAGNNRKLIW       1.0
CALSKGAAGNKLTF          1.0
CAMKSTNAGKSTF           1.0
CAMRAIKAAGNKLTF         1.0
CAMRDTGRRALTF           1.0
CAMSASNFGNEKLTF         1.0
CARGGGFKTIF             1.0
CAVEGMDTGRRALTF         1.0
CAVFPKAGGTSYGKLTF       1.0
CAVGDNFNKFYF            6.0
CAVHGYGQNFVF            1.0
CAVLNQAGTALIF           1.0
CAVRDHSGNTPLVF          1.0
CILRDLNAGGTSYGKLTF      1.0
CNPGGYNKLIF             1.0
CTSVQ*ERFILTGGGNKLTF    1.0
CVGYTNAGKSTF            1.0
CVVSGDTGRRALTF          1.0
None                    1.0
Name: umis, dtype: float64

In [30]:
clono2[clono2.cdr3s_aa.str.contains('CASSTSTGTGYGYTF')].shape

(42, 5)

In [176]:
df[df.gem.str.contains('AAGGCAGAGAGGGATA')]

,Unnamed: 0,gem,clonotype,num_clonotype,ct,genes_TRA,genes_TRB,genes_lst_TRA,genes_lst_TRB,length_TRA,...,normalized dex. Sig.,gem_icon,HLA_cd8_icon,peptide_HLA_icon,HLA_mhc_icon,v_alpha_match,j_alpha_match,v_beta_match,j_beta_match,peptide_HLA_match
6843,5917,AAGGCAGAGAGGGATA-33-s1,clonotype2410,2410,2410,TRAV12-2;TRAJ45;TRAC,TRBV5-4;None;TRBJ1-1;TRBC1,['TRAV12-2;TRAJ45;TRAC'],['TRBV5-4;TRBD2;TRBJ1-5;TRBC1' 'TRBV5-4;None;T...,555.0,...,2.992117,AAGGCAGAGAGGGATA-12,"['A0201', 'A1101', 'B3501']",IVTDFSVIK A1101,A1101,False,False,False,False,False


In [104]:
df.loc[(df.sample_id == 's1') & (df.cdr3_TRA == 'CALSEADDSWGKLQF'),
       ['gem','gem_icon','cdr3_TRA','alpha_cdr3','cdr3_TRB','beta_cdr3','clonotype','ct']].dropna(subset=['gem_icon','cdr3_TRA','cdr3_TRB'])

,gem,gem_icon,cdr3_TRA,alpha_cdr3,cdr3_TRB,beta_cdr3,clonotype,ct
283,AAACCTGTCACGATGT-11-s1,AAACCTGTCACGATGT-20,CALSEADDSWGKLQF,CAGHTGNQFYF,CASSLYGNLGTGELFF,CASSWGGGSHYGYTF,clonotype560,560


## Both chains annotated, but TRA is different

In [229]:
total_imps = 0
wrong_imps = 0
#for smpl,grp in df[(df.cdr3_TRA!=df.alpha_cdr3) &
#                   (df.cdr3_TRB==df.beta_cdr3)].dropna(subset=['gem_icon','cdr3_TRA','cdr3_TRB']).groupby('sample_id'):
for smpl,grp in tmp[(tmp.cdr3_TRA!=tmp.alpha_cdr3) &
                   (tmp.cdr3_TRB==tmp.beta_cdr3)].dropna(subset=['gem_icon']).dropna(subset=['cdr3_TRA','cdr3_TRB']).groupby('sample_id'):
    print(smpl)
    clono = get_clono()
    total_imps += len(grp)
    #for c_query, c_missing, cdr3_missing in zip(['TRA','TRB'], ['TRB','TRA'],['beta_cdr3','alpha_cdr3']):
    reliable = list()
    for name, row in grp.drop_duplicates(subset=['cdr3_TRA','cdr3_TRB','alpha_cdr3','beta_cdr3']).iterrows():
        dups = clono.cdr3s_aa.str.contains(row.cdr3_TRB)
        trxs = sum(dups & clono.cdr3s_aa.str.contains(row.alpha_cdr3))
        if (sum(dups) <= 2) & (trxs == 1):
            reliable.append(row.cdr3_TRB)
    wrong_imps += sum(~grp.cdr3_TRB.isin(reliable))
    print(reliable)

s1
['CASTRVGSTEAFF', 'CASSPPVGNEQFF', 'CASSSPGGMETQYF', 'CASSQDLAGGLKFF', 'CASSLIDAGELFF', 'CASSLTGAGYEQYF', 'CASSLWDNTEAFF', 'CASSTRSQETQYF', 'CASSPFGSDEQFF', 'CSARDETGNTIYF', 'CASSIRSNTEAFF', 'CASSTQTGSGYTF', 'CASAGGNQPQHF', 'CASSTGVYGYTF', 'CASSLGGARNTEAFF', 'CASSLTGQGAYGYTF', 'CASSFRLAGTGELFF', 'CSATLTSGGIYEQYF', 'CAASGRASETQYF', 'CASSDRSNTEAFF', 'CASSLGGLYLYNEQFF', 'CASSFGGRALPSSYNEQFF', 'CSARRQRSTDTQYF', 'CASSTRASEGQYF', 'CASSYPWGGASGYTF', 'CASSLASGATYQETQYF', 'CASSANRDTEDTGELFF', 'CASSIHKNQPQHF', 'CASSLRGDRGDTQYF', 'CASSEGDSNYEQYF', 'CASSVQGVLNTEAFF', 'CASSSPRLSGEADGYTF', 'CASSSRAGGEQYF', 'CASSLRGDNQPQHF', 'CASSLGTVHQPQHF']
s2
['CASSLRDRVHEQYF', 'CATSDQKTGSTPGELFF', 'CASSQVRGGYEQFF', 'CSAPPGDHTEAFF', 'CASSIRASVEQYF', 'CSAGQGLAGGPGTDTQYF', 'CASSFPVWGNEQFF', 'CASSFGGGTYEQYF', 'CASTSHGGANNEQFF', 'CASSGDSTNSPLHF', 'CASSTRLSGGTSNEQFF', 'CASSGKDRTEAFF', 'CASSLIGELFF', 'CSARGSGGDPTDTQYF', 'CASSEQGQGEKLFF', 'CASSSPGGFGYNSPLHF', 'CASSIFGQVDAFF', 'CASSNSGANVLTF', 'CASSGRAGDTQYF']
s3
['CAS

## Both chains annotated, but TRB is different

In [230]:
# OBS! Continuation of above!
#total_imps = 0
#wrong_imps = 0
#for smpl,grp in df[(df.cdr3_TRA==df.alpha_cdr3) &
#                   (df.cdr3_TRB!=df.beta_cdr3)].dropna(subset=['gem_icon','cdr3_TRA','cdr3_TRB']).groupby('sample_id'):
for smpl,grp in tmp[(tmp.cdr3_TRA==tmp.alpha_cdr3) &
                   (tmp.cdr3_TRB!=tmp.beta_cdr3)].dropna(subset=['gem_icon']).dropna(subset=['cdr3_TRA','cdr3_TRB']).groupby('sample_id'):
    print(smpl)
    clono = get_clono()
    total_imps += len(grp)
    #for c_query, c_missing, cdr3_missing in zip(['TRA','TRB'], ['TRB','TRA'],['beta_cdr3','alpha_cdr3']):
    reliable = list()
    for name, row in grp.drop_duplicates(subset=['cdr3_TRA','cdr3_TRB','alpha_cdr3','beta_cdr3']).iterrows():
        dups = clono.cdr3s_aa.str.contains(row.cdr3_TRA)
        trxs = sum(dups & clono.cdr3s_aa.str.contains(row.beta_cdr3))
        if (sum(dups) <= 2) & (trxs == 1):
            reliable.append(row.cdr3_TRA)
    wrong_imps += sum(~grp.cdr3_TRA.isin(reliable))
    print(reliable)

s1
['CAASASNQGGKLIF', 'CAASIEYGGSQGNLIF', 'CAARGGSNYQLIW', 'CAMFGMDSSYKLIF', 'CAMREGYGGKSTF', 'CAVTNYGGATNKLIF', 'CAVKGGNEKLTF', 'CAESIAGGTSYGKLTF', 'CAVQARGTGGFKTIF']
s2
['CALSEAPGAPDGLTF', 'CAASGTSNSGYALNF', 'CALMGNAGGTSYGKLTF', 'CALNNNARLMF', 'CAYRISLNNNDMRF', 'CAAISGAGSYQLTF', 'CVVSDLGYKLSF', 'CAMREAYGGSQGNLIF', 'CAFDQRSGGSNYKLTF', 'CILSPAGTASKLTF', 'CAYPTGNQFYF', 'CAVRGSQGNLIF', 'CAENDHNNNDMRF', 'CAASIKLNSGYALNF']
s3
['CAFPLNVNTGNQFYF']
s4
['CAAPNSGNTPLVF', 'CAVRDKDGGATNKLIF']


In [231]:
total_imps # tmp

1459

In [232]:
wrong_imps # tmp

1342

In [130]:
total_imps # old

1452

In [131]:
wrong_imps # old

1337

In [136]:
df.loc[(df.cdr3_TRA=='CAASIEYGGSQGNLIF') &
       (df.cdr3_TRB!=df.beta_cdr3),
       ['gem','gem_icon','clonotype','ct','cdr3_TRA','alpha_cdr3','cdr3_TRB','beta_cdr3']].dropna(subset=['gem_icon','cdr3_TRA','cdr3_TRB'])

,gem,gem_icon,clonotype,ct,cdr3_TRA,alpha_cdr3,cdr3_TRB,beta_cdr3
29858,AGGTCCGAGAATGTGT-36-s1,AGGTCCGAGAATGTGT-30,clonotype6012,6012,CAASIEYGGSQGNLIF,CAASIEYGGSQGNLIF,CASSFVDRAETQYF,CASSLDGTGTRNTEAFF


In [96]:
lol

,Unnamed: 0,gem,clonotype,num_clonotype,ct,genes_TRA,genes_TRB,genes_lst_TRA,genes_lst_TRB,length_TRA,...,normalized dex. Sig.,gem_icon,HLA_cd8_icon,peptide_HLA_icon,HLA_mhc_icon,v_alpha_match,j_alpha_match,v_beta_match,j_beta_match,peptide_HLA_match
46194,164519,CACATTTTCTGACCTC-8-s4,clonotype73745,73745,73745,NaN,TRBV19;TRBD2;TRBJ2-2;TRBC2,NaN,['TRBV19;TRBD2;TRBJ2-2;TRBC2'],NaN,...,8.621360,CACATTTTCTGACCTC-7,"['A0301', 'B0702', 'B5701']",KLGGALQAK A0301,A0301,False,False,False,False,True
80806,169008,CTAACTTAGTGTCTCA-5-s4,clonotype69117,69117,69117,NaN,TRBV24-1;TRBD2;TRBJ2-2;TRBC2,NaN,['TRBV24-1;TRBD2;TRBJ2-2;TRBC2'],NaN,...,2.630056,CTAACTTAGTGTCTCA-4,"['A0301', 'B0702', 'B5701']",IVTDFSVIK A1101,A1101,False,False,False,False,False
158068,178848,TCTATTGGTCCTCTTG-8-s4,clonotype83315,83315,83315,NaN,TRBV7-7;None;TRBJ2-1;TRBC2,NaN,['TRBV7-7;None;TRBJ2-1;TRBC2'],NaN,...,2.630056,TCTATTGGTCCTCTTG-7,"['A0301', 'B0702', 'B5701']",IVTDFSVIK A1101,A1101,False,False,False,False,False
162844,141894,TGAGCATGTACAGTGG-6-s4,clonotype69110,69110,69110,TRAV21;TRAJ50;TRAC,NaN,['TRAV12-3;TRAJ8;TRAC' 'TRAV21;TRAJ50;TRAC'],NaN,559.0,...,2.630056,TGAGCATGTACAGTGG-5,"['A0301', 'B0702', 'B5701']",IVTDFSVIK A1101,A1101,True,True,False,False,True
164106,179617,TGATTTCTCGTTGACA-7-s4,clonotype69229,69229,69229,NaN,TRBV28;TRBD1;TRBJ1-2;TRBC1,NaN,['TRBV28;TRBD1;TRBJ1-2;TRBC1'],NaN,...,2.630056,TGATTTCTCGTTGACA-2,"['A0301', 'B0702', 'B5701']",IVTDFSVIK A1101,A1101,False,False,False,False,False
164399,179656,TGCACCTTCACGATGT-3-s4,clonotype83888,83888,83888,NaN,TRBV30;None;TRBJ2-7;TRBC2,NaN,['TRBV30;None;TRBJ2-7;TRBC2'],NaN,...,2.630056,TGCACCTTCACGATGT-6,"['A0301', 'B0702', 'B5701']",IVTDFSVIK A1101,A1101,False,False,False,False,False
177925,181376,TTGCCGTCATTAGCCA-4-s4,clonotype69145,69145,69145,NaN,TRBV20-1;TRBD2;TRBJ2-7;TRBC2,NaN,['TRBV20-1;TRBD2;TRBJ2-7;TRBC2'],NaN,...,2.630056,TTGCCGTCATTAGCCA-7,"['A0301', 'B0702', 'B5701']",IVTDFSVIK A1101,A1101,False,False,False,False,False


In [52]:
clono1

,clonotype_id,frequency,proportion,cdr3s_aa,cdr3s_nt
0,clonotype1,2407,0.050583,TRA:CAGHTGNQFYF;TRB:CASSWGGGSHYGYTF,TRA:TGTGCTGGTCACACCGGTAACCAGTTCTATTTT;TRB:TGTG...
1,clonotype2,1244,0.026143,TRA:CAARVRGFGNVLHC;TRA:CAVGDNFNKFYF;TRB:CASSLY...,TRA:TGTGCAGCAAGAGTGCGGGGCTTTGGGAATGTGCTGCATTGC...
2,clonotype3,1167,0.024525,TRB:CASSWGGGSHYGYTF,TRB:TGTGCCAGCAGCTGGGGGGGCGGTAGCCACTATGGCTACACCTTC
3,clonotype4,512,0.010760,TRA:CAVSAASGGSYIPTF;TRB:CASSPRDRERGEQYF,TRA:TGTGCTGTGAGTGCAGCATCAGGAGGAAGCTACATACCTACA...
4,clonotype5,419,0.008805,TRA:CAMNPAWGGATNKLIF;TRB:CSASPGDYEQYF,TRA:TGTGCAATGAACCCGGCGTGGGGTGGTGCTACAAACAAGCTC...
...,...,...,...,...,...
30335,clonotype22413,1,0.000021,TRB:CSARDRDYGYTF,TRB:TGCAGCGCCCGGGACAGGGACTATGGCTACACCTTC
30336,clonotype22410,1,0.000021,TRB:CSVASYKNTGELFF,TRB:TGCAGCGTTGCATCCTACAAGAACACCGGGGAGCTGTTTTTT
30337,clonotype22411,1,0.000021,TRA:CAVNDDTDKLIF;TRB:CASSFGQNSNQPQHF,TRA:TGTGCTGTGAATGATGACACCGACAAGCTCATCTTT;TRB:T...
30338,clonotype22418,1,0.000021,TRA:CAIDLRSAGGTSYGKLTF;TRB:CASSLSVIAGGRETQYF,TRA:TGTGCAATTGATCTACGGAGTGCTGGTGGTACTAGCTATGGA...


In [80]:
df.columns.to_list()

['Unnamed: 0',
 'gem',
 'clonotype',
 'num_clonotype',
 'ct',
 'genes_TRA',
 'genes_TRB',
 'genes_lst_TRA',
 'genes_lst_TRB',
 'length_TRA',
 'cdr3_TRA',
 'umi_count_TRA',
 'umi_count_lst_TRA',
 'delta_umi_TRA',
 'cdr3_lst_TRA',
 'chain_count_TRA',
 'single_TRA',
 'length_TRB',
 'cdr3_TRB',
 'umi_count_TRB',
 'umi_count_lst_TRB',
 'delta_umi_TRB',
 'cdr3_lst_TRB',
 'chain_count_TRB',
 'single_TRB',
 'single_chain_only',
 'umi_count_tcr',
 'cdr3_comb',
 'v_gene_TRA',
 'j_gene_TRA',
 'v_gene_TRB',
 'j_gene_TRB',
 'tcr_category',
 'no_filtration',
 'exclude_single-chain_TCRs',
 'exclude_ambiguous_and_single-chain_TCRs',
 'exclude_ambiguous_TCRs',
 'label',
 'cell_flag',
 'cell_high_confidence',
 'umi_count_mhc',
 'umi_count_lst_mhc',
 'delta_umi_mhc',
 'single_barcode_mhc',
 'multiplets_mhc',
 'umi_count_cd8',
 'umi_count_lst_cd8',
 'delta_umi_cd8',
 'single_barcode_cd8',
 'multiplets_cd8',
 'detected_response',
 'peptide_assayed',
 'sample_id',
 'sample_id_lst',
 'HLA_pool_cd8',
 'HLA_ls